# Keras Tuner
https://www.tensorflow.org/tutorials/keras/keras_tuner?hl=ko

In [28]:
tensorflow.__version__

'2.5.0'

In [29]:
keras.__version__

'2.5.0'

In [33]:
import tensorflow as tf
from tensorflow import keras
import IPython

# !pip install -q -U keras-tuner
import keras_tuner as kt

import pandas as pd
import numpy as np
import warnings
warnings.filterwarnings(action='ignore')

from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, LSTM
from tensorflow.keras.layers import SimpleRNN
from tensorflow.keras.layers import Dropout, InputLayer
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras.regularizers import l2
from tensorflow.keras.optimizers import Adam

In [34]:
import random    
random.seed(seed_num)

x = np.load('/project/LSH/x_(7727,10,4069).npy')
y = np.load('/project/LSH/y_(7727,1).npy')

idx = list(range(len(x)))
random.shuffle(idx)

i = round(x.shape[0]*0.8)
X_train, y_train = x[idx[:i],:,:], y[idx[:i]]
X_test, y_test = x[idx[i:],:,:], y[idx[i:]]

X_train.shape, y_train.shape, X_test.shape, y_test.shape

((6182, 10, 4069), (6182,), (1545, 10, 4069), (1545,))

In [35]:
def model_builder(hp):
    model = Sequential()
    model.add(InputLayer(input_shape=(X_train.shape[1],X_train.shape[2])))
    
    # Tune the number of units in the first Dense layer
    # Choose an optimal value between 32-512
    hp_units = hp.Int('units', min_value = 32, max_value = 512, step = 32)
    model.add(LSTM(units = hp_units, activation = 'hard_sigmoid', return_sequences=True))
    model.add(LSTM(units = hp_units, activation = 'hard_sigmoid', return_sequences=True))
    model.add(Dropout(0.2))
    model.add(LSTM(units = hp_units, activation = 'hard_sigmoid', return_sequences=True))
    model.add(LSTM(units = hp_units, activation = 'hard_sigmoid', return_sequences=False))
    model.add(Dropout(0.2))
    model.add(Dense(1, activation='sigmoid'))

    # Tune the learning rate for the optimizer 
    # Choose an optimal value from 0.01, 0.001, or 0.0001
    hp_learning_rate = hp.Choice('learning_rate', values = [1e-2, 1e-3, 1e-4]) 

    model.compile(optimizer = keras.optimizers.Adam(learning_rate = hp_learning_rate),
                loss = "binary_crossentropy", 
                metrics = ['accuracy'])
    
    return model

In [36]:
ep = 500
pa = 10
early_stop = EarlyStopping(monitor='val_acc', patience=pa, verbose=1, restore_best_weights=True)
tuner = kt.Hyperband(model_builder,
                     objective = 'val_accuracy', 
                     max_epochs = ep,
                     factor = 3,
                     directory = 'my_dir',
                     project_name = 'intro_to_kt')

INFO:tensorflow:Reloading Oracle from existing project my_dir/intro_to_kt/oracle.json


In [37]:
class ClearTrainingOutput(tf.keras.callbacks.Callback):
    def on_train_end(*args, **kwargs):
        IPython.display.clear_output(wait = True)

In [39]:
# tuner.search(X_train, y_train, epochs = 10, validation_data = (X_test, y_test), callbacks = [ClearTrainingOutput()])

# # Get the optimal hyperparameters
# best_hps = tuner.get_best_hyperparameters(num_trials = 1)[0]

# print(f"""
#         The hyperparameter search is complete. The optimal number of units in the first densely-connected
#         layer is {best_hps.get('units')} and the optimal learning rate for the optimizer
#         is {best_hps.get('learning_rate')}.
#         """)

In [ ]:
# Build the model with the optimal hyperparameters and train it on the data
model = tuner.hypermodel.build(best_hps)
model.fit(X_train, y_train, epochs = 500, validation_data = (X_test, y_test), callbacks=[early_stop], shuffle=False)

## Deeper and Deeper

In [41]:
def model_builder(hp):
    model = Sequential()
    model.add(InputLayer(input_shape=(X_train.shape[1],X_train.shape[2])))
    
    # Tune the number of units in the first Dense layer
    # Choose an optimal value between 32-512
    hp_units1 = hp.Int('units1', min_value = 32, max_value = 512, step = 32)
    hp_units2 = hp.Int('units2', min_value = 32, max_value = 512, step = 32)
    hp_units3 = hp.Int('units3', min_value = 32, max_value = 512, step = 32)
    hp_units4 = hp.Int('units4', min_value = 32, max_value = 512, step = 32)
    
    hp_activ1 = hp.Choice('activation1', values=['relu', 'tanh', 'hard_sigmoid'], default='hard_sigmoid')
    hp_activ2 = hp.Choice('activation2', values=['relu', 'tanh', 'hard_sigmoid'], default='hard_sigmoid')
    hp_activ3 = hp.Choice('activation3', values=['relu', 'tanh', 'hard_sigmoid'], default='hard_sigmoid')
    hp_activ4 = hp.Choice('activation4', values=['relu', 'tanh', 'hard_sigmoid'], default='hard_sigmoid')
    
    hp_drop1 = hp.Float('dropout1', min_value=0.0,max_value=0.5,default=0.2,step=0.05)
    hp_drop2 = hp.Float('dropout2', min_value=0.0,max_value=0.5,default=0.2,step=0.05)
    
    hp_activ_dense = hp.Choice('dense_activation', values=['relu', 'tanh', 'sigmoid'], default='sigmoid')
    #model
    model.add(LSTM(units = hp_units1, activation = hp_activ1, return_sequences=True))
    model.add(LSTM(units = hp_units2, activation = hp_activ2, return_sequences=True))
    model.add(Dropout(rate=hp_drop1))
    model.add(LSTM(units = hp_units3, activation = hp_activ3, return_sequences=True))
    model.add(LSTM(units = hp_units4, activation = hp_activ4, return_sequences=False))
    model.add(Dropout(rate=hp_drop2))
    model.add(Dense(1, activation=hp_activ_dense))

    # Tune the learning rate for the optimizer 
    # Choose an optimal value from 0.01, 0.001, or 0.0001
    hp_learning_rate = hp.Choice('learning_rate', values = [1e-2, 1e-3, 1e-4]) 

    model.compile(optimizer = keras.optimizers.Adam(learning_rate = hp_learning_rate),
                loss = "binary_crossentropy", 
                metrics = ['accuracy'])
    
    return model

In [42]:
ep = 500
pa = 10
early_stop = EarlyStopping(monitor='val_acc', patience=pa, verbose=1, restore_best_weights=True)
tuner = kt.Hyperband(model_builder,
                     objective = 'val_accuracy', 
                     max_epochs = ep,
                     factor = 3,
                     directory = 'my_dir',
                     project_name = 'intro_to_kt')

INFO:tensorflow:Reloading Oracle from existing project my_dir/intro_to_kt/oracle.json
INFO:tensorflow:Reloading Tuner from my_dir/intro_to_kt/tuner0.json


In [ ]:
%%time
tuner.search(X_train, y_train, epochs = 10, validation_data = (X_test, y_test), callbacks = [ClearTrainingOutput()])

# Get the optimal hyperparameters
best_hps = tuner.get_best_hyperparameters(num_trials = 1)[0]

print(f"""
        The hyperparameter search is complete. The optimal number of units in the first densely-connected
        layer is {best_hps.get('units')} and the optimal learning rate for the optimizer
        is {best_hps.get('learning_rate')}.
        """)

Trial 106 Complete [00h 01m 44s]
val_accuracy: 0.7864077687263489

Best val_accuracy So Far: 0.798705518245697
Total elapsed time: 01h 23m 59s

Search: Running Trial #107

Hyperparameter    |Value             |Best Value So Far 
units             |384               |384               
learning_rate     |0.0001            |0.001             
activation        |tanh              |hard_sigmoid      
dropout           |0                 |0.35              
dense_activation  |tanh              |sigmoid           
tuner/epochs      |3                 |3                 
tuner/initial_e...|0                 |0                 
tuner/bracket     |5                 |5                 
tuner/round       |0                 |0                 

Epoch 1/3
194/194 [==============================] - 33s 154ms/step - loss: 0.8279 - accuracy: 0.5718 - val_loss: 0.6372 - val_accuracy: 0.6291
Epoch 2/3
109/194 [===============>..............] - ETA: 11s - loss: 0.6257 - accuracy: 0.6692

In [ ]:
# Build the model with the optimal hyperparameters and train it on the data
model = tuner.hypermodel.build(best_hps)
model.fit(X_train, y_train, epochs = 500, validation_data = (X_test, y_test), callbacks=[early_stop], shuffle=False)